- CEFR(유럽 언어 공통 기준, Common European Framework of Reference for Languages) 기반 
- 사용자 어학수준 평가 기능

## 학습 데이터 로드
- 실제 해당 레벨의 학습자가 어떤 문장을 구사하는지 정리된 데이터셋

In [1]:
# 환경 설정 및 경고 비활성화
import warnings

warnings.filterwarnings("ignore")

import os
import numpy as np

# 프로젝트 캐시 설정
project_cache = "./models_cache"
os.environ["HF_HOME"] = project_cache
print(f"프로젝트 캐시 활성화: {os.path.abspath(project_cache)}")

# Tokenizers 병렬 처리 경고 비활성화
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# 캐시 디렉토리 확인
cache_dir = os.environ.get("HF_HOME", os.path.expanduser("~/.cache/huggingface"))
print(f"모델 저장 위치: {cache_dir}")

print("환경 설정 완료")

프로젝트 캐시 활성화: /Users/yuli/Documents/AI_Workspace/derdiedas-ai.ai/notebooks/models_cache
모델 저장 위치: ./models_cache
환경 설정 완료


In [3]:
from datasets import load_dataset

ds = load_dataset("UniversalCEFR/merlin_de", split="train")
print(ds)
print(ds[0])

print(f"\n=== UniversalCEFR 데이터셋 정보 ===")
print(f"총 샘플 수: {len(ds):,}개")
print(f"컬럼: {ds.column_names}")
print(f"\n첫 번째 샘플:")
print(f"  제목: {ds[0]['title']}")
print(f"  언어: {ds[0]['lang']}")
print(f"  카테고리: {ds[0]['category']}")
print(f"  레벨: {ds[0]['cefr_level']}")
print(f"  텍스트: {ds[0]['text']}")

print("\n데이터셋 메타정보 확인 완료")

Dataset({
    features: ['title', 'lang', 'source_name', 'format', 'category', 'cefr_level', 'license', 'text'],
    num_rows: 1033
})
{'title': '1023_0001416.txt', 'lang': 'de', 'source_name': 'merlin-de', 'format': 'paragraph-level', 'category': 'learner', 'cefr_level': 'B2', 'license': 'CC BY-SA 4.0', 'text': 'M. Meier\nMüllergasse 1\n12345 Stadt X\nInternationale Au-pair Vermittlung\nBahnhofstr. 101\n65185 Wiesbaden\n                                   Stadt X, den 14.05.11.\nSehr geehrte Damen und Herren,\nihre Anzeige in der Zeitung hat mich sehr erfreut, so was habe ich schon lange gesucht. \nDa mir die Arbeit mit anderen Menschen sehr großen Spas bereitet, habe ich mich entschlossen, mich zu bewerben.\nIch bin flexibel und für neuen Aufgaben ofen.\nTakt, Geduld, warmherzigkeit und Zuverlässigkeit bringe ich mit, sowie Erfahrung im Umgang mit Kindern und der Haushaltsführung.\nIch spreche Russisch, Englisch, Kasachisch, habe Grundkenntnisse in Deutsch. Zusätzlich habe ich Compute

In [4]:
import pysbd
seg = pysbd.Segmenter(language="de", clean=True)
def split_sentences(text: str):
    sents = seg.segment(text)
    # 길이 필터 등 후처리
    return [s.strip() for s in sents if 5 <= len(s.split()) <= 80]


In [5]:
# 슬라이딩 윈도우 생성 (2~3문장)
def make_windows(sents, win=2, stride=1):
    out = []
    for i in range(0, len(sents) - win + 1, stride):
        out.append(" ".join(sents[i : i + win]))
    return out

In [6]:
def doc_to_instances_windows_only(
    doc_id: str,
    text: str,
    doc_label: str,
    split_sentences,
    make_windows,
    wins=(2, 3),
    stride=1,
    min_tokens=5,
    max_tokens=120,
    add_numeric_level=True
):
    """
    문장은 저장하지 않고, 2~3문장 윈도우만 생성하여 반환.
    - doc_label은 window 레벨에도 그대로 전파 (label_doc 유지)
    - add_numeric_level=True면 레벨을 수치화한 label_num도 추가 (예: A1=1, A1+=1.5, A2=2, ...)
    """
    def _tok_len(t: str) -> int:
        return len(t.split())

    # CEFR 레벨 수치화(선택)
    level2num = {
        "A1":1.0, "A1+":1.5,
        "A2":2.0, "A2+":2.5,
        "B1":3.0, "B1+":3.5,
        "B2":4.0, "B2+":4.5,
        "C1":5.0, "C2":6.0
    }
    label_num = level2num.get(doc_label, None) if add_numeric_level else None

    instances = []
    sents = split_sentences(text)

    for w in wins:
        wins_out = make_windows(sents, win=w, stride=stride)
        for j, item in enumerate(wins_out):
            # make_windows가 (span_idxs, text) 또는 text 만 반환하는 두 케이스 모두 지원
            if isinstance(item, tuple) and len(item) == 2:
                span, wtext = item
            else:
                span, wtext = None, item
            if min_tokens <= _tok_len(wtext) <= max_tokens:
                row = {
                    "doc_id": doc_id,
                    "unit": f"win{w}",
                    "idx": j,
                    "text": wtext,
                    "label_doc": doc_label,     # 문서 라벨 유지
                    "span": span                # 원 문장 인덱스 범위 (있을 경우)
                }
                if label_num is not None:
                    row["label_num"] = label_num
                instances.append(row)
    return instances


In [7]:
import os
import pandas as pd
from datasets import load_dataset

# 네 구현체 import (split_sentences, make_windows)
# from your_module import split_sentences, make_windows

def process_to_windows_only(
    ds_iter,
    text_key: str = "text",
    label_key: str = "cefr_level",
    id_key: str | None = None,
    max_docs: int | None = None,
    **doc_kwargs
) -> pd.DataFrame:
    """
    HF dataset iterable을 받아 윈도우 인스턴스만 DataFrame으로 변환
    """
    all_rows = []
    for k, rec in enumerate(ds_iter):
        if max_docs is not None and k >= max_docs:
            break
        doc_id = rec[id_key] if id_key and id_key in rec else f"doc_{k}"
        text = rec[text_key]
        label = rec[label_key]
        rows = doc_to_instances_windows_only(
            doc_id=doc_id,
            text=text,
            doc_label=label,
            split_sentences=split_sentences,
            make_windows=make_windows,
            **doc_kwargs
        )
        all_rows.extend(rows)

    df = pd.DataFrame(all_rows)
    if not df.empty and {"doc_id","unit","idx"}.issubset(df.columns):
        df = df.sort_values(["doc_id","unit","idx"]).reset_index(drop=True)
    return df

def save_parquet_jsonl(df: pd.DataFrame, base_path: str):
    os.makedirs(os.path.dirname(base_path), exist_ok=True)
    pq = base_path + ".parquet"
    jl = base_path + ".jsonl"
    df.to_parquet(pq, index=False)
    with open(jl, "w", encoding="utf-8") as f:
        for r in df.to_dict(orient="records"):
            import json
            f.write(json.dumps(r, ensure_ascii=False) + "\n")
    print(f"✅ Saved: {pq} | rows={len(df):,}")
    print(f"✅ Saved: {jl} | rows={len(df):,}")


In [8]:
from statistics import mean

def verify_windows_df(df: pd.DataFrame, preview=8):
    assert set(df["unit"].unique()) <= {"win2","win3"}, "⚠️ sent 행이 섞여 있습니다. (unit 컬럼 확인)"

    print("🧾 Preview")
    print(df[["doc_id","unit","idx","label_doc","text"]].head(preview))

    print("\n📊 Unit counts")
    print(df["unit"].value_counts())

    # 길이 통계
    lens = df["text"].str.split().apply(len)
    print("\n📈 Token length stats")
    print(f"- mean: {lens.mean():.1f}, min: {lens.min()}, max: {lens.max()}")

    # 라벨 분포
    print("\n🏷️ Label distribution (label_doc)")
    print(df["label_doc"].value_counts())

    # 문서별 커버리지 샘플
    sample_doc = df["doc_id"].iloc[0]
    print(f"\n👀 Sample windows for doc_id={sample_doc}")
    print(df[df["doc_id"] == sample_doc].head(10)[["unit","idx","text"]])


## 분할된 데이터 검토

In [18]:
# 1) 데이터 로드
ds = load_dataset("UniversalCEFR/merlin_de", split="train")

# 2) 윈도우 인스턴스 생성 (문장 저장 X)
df_win = process_to_windows_only(
    ds_iter=ds,
    text_key="text",
    label_key="cefr_level",
    id_key=None,          # 데이터에 고유 id 있으면 지정
    max_docs=None,        # 일부만 테스트하려면 숫자
    wins=(2,3),           # 2~3문장 윈도우
    stride=3,         # 중복 설정
    min_tokens=5,
    max_tokens=120,
    add_numeric_level=True
)

# 3) 저장
base = "outputs/parsed/ucefr_elg_en_windows_only"
save_parquet_jsonl(df_win, base)

# 4) 검증
verify_windows_df(df_win, preview=10)



✅ Saved: outputs/parsed/ucefr_elg_en_windows_only.parquet | rows=5,996
✅ Saved: outputs/parsed/ucefr_elg_en_windows_only.jsonl | rows=5,996
🧾 Preview
  doc_id  unit  idx label_doc  \
0  doc_0  win2    0        B2   
1  doc_0  win2    1        B2   
2  doc_0  win2    2        B2   
3  doc_0  win3    0        B2   
4  doc_0  win3    1        B2   
5  doc_0  win3    2        B2   
6  doc_1  win2    0        B2   
7  doc_1  win2    1        B2   
8  doc_1  win2    2        B2   
9  doc_1  win2    3        B2   

                                                text  
0  Sehr geehrte Damen und Herren, ihre Anzeige in...  
1  Ich bin flexibel und für neuen Aufgaben ofen. ...  
2  Naturlich würde ich gern Deutsche Sprache und ...  
3  Sehr geehrte Damen und Herren, ihre Anzeige in...  
4  Ich bin flexibel und für neuen Aufgaben ofen. ...  
5  Naturlich würde ich gern Deutsche Sprache und ...  
6  Sehr geehrte Damen und Herren ich habe Ihre An...  
7  Für diese Stelle bringe ich alle Voraussetz

In [ ]:
# df_win